In [1]:
# installs
# !pip install bertopic 

# imports
import pandas as pd
import os
from bertopic import BERTopic
import nltk
from nltk.corpus import stopwords
# from bertopic import preprocess_text
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from tokenizers.pre_tokenizers import Whitespace, Punctuation
from tokenizers import pre_tokenizers
# nltk.download('stopwords')
import gzip

# !wget https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFilesSmall/Books_5.json.gz
# !wget https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFilesSmall/Home_and_Kitchen_5.json.gz
# !wget https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFilesSmall/Sports_and_Outdoors_5.json.gz
# !wget https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFilesSmall/Electronics_5.json.gz
    
print("imports compelete")


/data/user/anna19/Conda_Env/nlp2023v2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


imports compelete


In [2]:
%%time

# read in data

datafile1 = "data/" + "Gift_Cards_5.json"
datafile2 = "data/" + "Industrial_and_Scientific_5.json"

review_text = []

# with open(datafile1, "r") as f:
#     data = f.readlines()
#     for review in data:
#         text = json.loads(review).get("reviewText", "").strip()
#         summary = json.loads(review).get("summary", "").strip()
#         review = summary + " " + text
#         if review.strip():
#             review_text.append(review)
            
# with open(datafile2, "r") as f:
#     data = f.readlines()
#     for review in data:
#         text = json.loads(review).get("reviewText", "").strip()
#         summary = json.loads(review).get("summary", "").strip()
#         review = summary + " " + text
#         if review.strip():
#             review_text.append(review)
            
# with gzip.open("Books_5.json.gz") as f:
#     for review in f:
#         text = json.loads(review).get("reviewText", "").strip()
#         summary = json.loads(review).get("summary", "").strip()
#         review = summary + " " + text
#         if review.strip():
#             review_text.append(review)

training_data_books = []
training_data_outdoors = []
training_data_electronics = []
training_data_kitchen=[]
with gzip.open('Books_5.json.gz') as f:
    for review in f:
        text = json.loads(review).get("reviewText", "").strip()
        summary = json.loads(review).get("summary", "").strip()
        review = summary + " " + text
        if review.strip():
            training_data_books.append(review)

# with gzip.open('Home_and_Kitchen_5.json.gz') as f:
#     for review in f:
#         text = json.loads(review).get("reviewText", "").strip()
#         summary = json.loads(review).get("summary", "").strip()
#         review = summary + " " + text
#         if review.strip():
#             review_text.append(review)

# with gzip.open('Sports_and_Outdoors_5.json.gz') as f:
#     for review in f:
#         text = json.loads(review).get("reviewText", "").strip()
#         summary = json.loads(review).get("summary", "").strip()
#         review = summary + " " + text
#         if review.strip():
#             review_text.append(review)

# with gzip.open('Electronics_5.json.gz') as f:
#     for review in f:
#         text = json.loads(review).get("reviewText", "").strip()
#         summary = json.loads(review).get("summary", "").strip()
#         review = summary + " " + text
#         if review.strip():
#             review_text.append(review)

# complete_test=[]
# complete_test.extend(training_data_books[:int(0.8*len(training_data_books))])
# complete_test.extend(training_data_electronics[:int(0.8*len(training_data_electronics))])
# complete_test.extend(training_data_outdoors[:int(0.8*len(training_data_outdoors))])
# complete_test.extend(training_data_kitchen[:int(0.8*len(training_data_kitchen))])
    
            
# print("data read in")
# print(len(review_text))
# print(review_text[3])
with gzip.open('Home_and_Kitchen_5.json.gz') as f:
    for review in f:
        text = json.loads(review).get("reviewText", "").strip()
        summary = json.loads(review).get("summary", "").strip()
        review = summary + " " + text
        if review.strip():
            training_data_kitchen.append(review)
            
print("done")

with gzip.open('Sports_and_Outdoors_5.json.gz') as f:
    for review in f:
        text = json.loads(review).get("reviewText", "").strip()
        summary = json.loads(review).get("summary", "").strip()
        review = summary + " " + text
        if review.strip():
            training_data_outdoors.append(review)
            
print("done")

done
done
CPU times: user 9min 39s, sys: 11.2 s, total: 9min 50s
Wall time: 9min 51s


In [10]:
%%time


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



A great tool for working with Snow Crab legs! More powerful, yet more tactile than the stainless steel type. I was always cracking the claws too aggressively with those and leaving a lot of shell shards to peel off. With this orange boy I can put just the right amount of pressure to let me crack the shell and get to the chunk of deliciousness inside. I wish I had gotten more of these. Everyone wants to use it and I don't like waiting to get my cracker back when I'm in Snow Crab mode.
We just moved to Florida and my husband is constantly coming home with crab that he catches. Pliers don't work so well, duh, so I got on line and found these. Husband came home with 25 crabs and the Zyliss seafood cracker and OXO pickers arrived on the same day. I wasn't sure what to expect, but they worked excellent. Highly recommend!
Works good.  Product as described
excellent product - work beautifully for dungeness.  also work great for opening bottle of tonic to go with the gin!
Couldn't ask for more!

KeyboardInterrupt: 

done


In [7]:
with gzip.open('Electronics_5.json.gz') as f:
    for review in f:
        text = json.loads(review).get("reviewText", "").strip()
        summary = json.loads(review).get("summary", "").strip()
        review = summary + " " + text
        if review.strip():
            training_data_electronics.append(review)
print("done")

EOFError: Compressed file ended before the end-of-stream marker was reached

In [9]:
complete_test=[]
# complete_test.extend(training_data_books[:int(0.8*len(training_data_books))])
# complete_test.extend(training_data_electronics[:int(0.8*len(training_data_electronics))])
complete_test.extend(training_data_outdoors[:int(0.8*len(training_data_outdoors))])
complete_test.extend(training_data_kitchen[:int(0.8*len(training_data_kitchen))])
    
            
print("data read in")
print(len(training_data_books))
print(len(complete_test))
print(complete_test[3])

data read in
27164817
7790987
I liked that the elastic waist didn't dig in (no muffin ... I used this skirt for a Halloween costume and glued a bunch of feathers over it.  It was exactly what I needed.  I liked that the elastic waist didn't dig in (no muffin top).  Two things to note though:  (i) it is a little long; I cut about 8 inches off, but it was really easy to do; and (ii) it is see-thru b/c unlined - I had to get a pair of yellow shorts to wear underneath which I hadn't planned on doing.


In [20]:
# preprocess data

# pre_tokenizer = pre_tokenizers.Sequence([Whitespace(), Punctuation()]) 

# review_data_pretok = [[word[0] for word in pre_tokenizer.pre_tokenize_str(review)] for review in review_text]

# print(review_data_pretok[0])

# def remove_stopwords(corpus):
#     # Reference: https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
#     stop_words = set(stopwords.words('english'))
#     corpus = [[token for token in doc if token.lower() not in stop_words] for doc in corpus]
#     return corpus

# reviews_cleaned = remove_stopwords(review_data_pretok)

# print(reviews_cleaned[0])



['Five', 'Stars', 'Another', 'great', 'gift', '.']
['Five', 'Stars', 'Another', 'great', 'gift', '.']


In [ ]:
%%time
# train and save model
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
print(complete_test[:5])

cluster_model = KMeans(n_clusters=200, n_init=2, max_iter=100, random_state=42)

# we add this to remove stopwords
vectorizer_model = CountVectorizer(ngram_range=(1, 1), stop_words="english", min_df = 10)

model = BERTopic(
    vectorizer_model=vectorizer_model,
    hdbscan_model=cluster_model,
    language='english', calculate_probabilities=True,
    verbose=True
)

# print(len(review_text))
topics, probabilities = model.fit_transform(complete_test)

model.save("bertopic_model_large")

print("model done")

['Five Stars What a spectacular tutu! Very slimming.', 'Is this a tutu for nuns?! What the heck? Is this a tutu for nuns? I know you can cut it but STILL. Also there aren\'t several layers of the tutu making it "poof out" It just lays flat. Needless to say it was returned.', 'Five Stars Exactly what we were looking for!', "I liked that the elastic waist didn't dig in (no muffin ... I used this skirt for a Halloween costume and glued a bunch of feathers over it.  It was exactly what I needed.  I liked that the elastic waist didn't dig in (no muffin top).  Two things to note though:  (i) it is a little long; I cut about 8 inches off, but it was really easy to do; and (ii) it is see-thru b/c unlined - I had to get a pair of yellow shorts to wear underneath which I hadn't planned on doing.", "This is thick enough that you can't see through it ... This is thick enough that you can't see through it, but it is very long, so be sure to check the dimensions. I ended up cutting it shorter."]


Batches: 100%|██████████| 243469/243469 [1:02:41<00:00, 64.72it/s] 
2023-11-20 19:50:31,786 - BERTopic - Transformed documents to Embeddings


In [86]:
# visualize results

model.visualize_topics()

# model.get_topic_info()

In [87]:
print(model.get_topic_info()[:11])
top_words = model.get_topics()

# Print the top words for each topic
for topic_id, words in top_words.items():
    words = [word[0] for word in words]
    print(f"Topic {topic_id}: {', '.join(words)}")

# # Assign topics to documents
# for i, topic in enumerate(topics):
#     print(f"Document {i + 1} is assigned to Topic {topic}")

    Topic  Count                                   Name   
0       0   7715        0_filament_printer_print_prints  \
1       1   6801               1_bottles_use_oil_bottle   
2       2   6267             2_quality_good_price_great   
3       3   5918                3_drill_tool_wheels_cut   
4       4   5174  4_accurate_meter_humidity_temperature   
5       5   4906               5_stars_works_great_work   
6       6   3694             6_product_good_works_great   
7       7   3662           7_screws_magnets_screw_hooks   
8       8   3384                8_glue_stuff_epoxy_used   
9       9   3367                9_tape_duct_roll_sticky   
10     10   3345         10_wire_shrink_connectors_heat   

                                       Representation   
0   [filament, printer, print, prints, pla, 3d, pr...  \
1   [bottles, use, oil, bottle, bags, water, great...   
2   [quality, good, price, great, work, nice, work...   
3   [drill, tool, wheels, cut, work, cutting, whee...   
4   [a